In [4]:
import pandas as pd

from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA, BollingerBands
from indicators import time_to_epoch, epoch_to_time
import datetime
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import requests

def get_market_rates(symbol:str, timeframe:str, previous_candle_count:int, eohlcv:bool=True, base_url:str="http://10.168.85.228:7767/data") -> np.ndarray:
    if timeframe not in ["1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"]:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')

    if timeframe == "1m":
        _tf = 1
    elif timeframe == "5m":
        _tf = 5
    elif timeframe == "15m":
        _tf = 15
    elif timeframe == "30m":
        _tf = 30
    elif timeframe == "1h":
        _tf = 16385
    elif timeframe == "1d":
        _tf = 16408
    elif timeframe == "4h":
        _tf = 16388
    elif timeframe == "1w":
        _tf = 32769
    elif timeframe == "1mo":
        _tf = 49153
    else:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')
    payload = {
        "symbol": symbol,
        "timeframe": _tf,
        "previous": previous_candle_count
    }

    response = requests.post(base_url, json=payload)

    if response.status_code == 200:
        _data = response.json()['rates']
        # print("Bars received:", len(_data))
        if eohlcv:
            return np.array(_data)[:, :6]
        else:
            return np.array(_data)
    else:
        print("Error:", response.text)
        raise RuntimeError(response.text)


In [44]:
data_1h = get_market_rates("XAUUSD", timeframe="1h", previous_candle_count=100).tolist()
data_4h = get_market_rates("XAUUSD", timeframe="4h", previous_candle_count=100).tolist()
data_1d = get_market_rates("XAUUSD", timeframe="1d", previous_candle_count=100).tolist()

current_epoch = data_1h[-1][0]
current_time = epoch_to_time(current_epoch, timezone="UTC")
current_price = data_1h[-1][4]
print(f"Current time is {current_time} and current price is {current_price}")


print("Following are the 100 previous candles on timeframe of 1 hour:")
for i in data_1h:
    print(str(i)[1:-1])
last_1h = i


print("Following are the 100 previous candles on timeframe of 4 hour:")
for i in data_4h:
    print(str(i)[1:-1])
last_4h = i
# print(epoch_to_time(i[0], timezone="UTC"))


print("Following are the 100 previous candles on timeframe of 1 day:")
for i in data_1d:
    print(str(i)[1:-1])
# print(epoch_to_time(i[0], timezone="UTC"))
last_1d = i


Current time is 2026-01-02 09:00:00 and current price is 4382.82
Following are the 100 previous candles on timeframe of 1 hour:
1766714400.0, 4493.2, 4517.85, 4493.2, 4517.74, 17176.0
1766718000.0, 4517.79, 4531.3, 4497.61, 4503.88, 33075.0
1766721600.0, 4503.88, 4509.84, 4492.5, 4508.7, 27391.0
1766725200.0, 4508.7, 4513.53, 4497.53, 4501.85, 23435.0
1766728800.0, 4501.85, 4511.69, 4500.55, 4509.01, 16671.0
1766732400.0, 4509.01, 4514.78, 4506.01, 4509.82, 22677.0
1766736000.0, 4509.97, 4519.45, 4508.1, 4511.65, 23643.0
1766739600.0, 4511.65, 4515.64, 4507.44, 4511.02, 16488.0
1766743200.0, 4511.02, 4519.43, 4509.74, 4518.78, 16080.0
1766746800.0, 4518.76, 4520.47, 4511.98, 4512.75, 13458.0
1766750400.0, 4512.75, 4523.66, 4511.33, 4514.46, 15780.0
1766754000.0, 4514.46, 4515.32, 4503.02, 4509.81, 22536.0
1766757600.0, 4509.84, 4514.95, 4506.78, 4508.8, 18597.0
1766761200.0, 4508.8, 4528.51, 4503.7, 4525.13, 30278.0
1766764800.0, 4525.15, 4533.48, 4512.89, 4521.07, 36603.0
1766768400.0

In [45]:
print("1Hr last candle:", epoch_to_time(last_1h[0], timezone="UTC"))
print("4Hr last candle:", epoch_to_time(last_4h[0], timezone="UTC"))
print("1D last candle:", epoch_to_time(last_1d[0], timezone="UTC"))

1Hr last candle: 2026-01-02 09:00:00
4Hr last candle: 2026-01-02 08:00:00
1D last candle: 2026-01-02 00:00:00


In [38]:
epoch_to_time(get_market_rates("XAUUSD", timeframe="1h", previous_candle_count=0).tolist()[-1][0], timezone="UTC")

'2026-01-02 06:00:00'

In [35]:
epoch_to_time(1767330000, timezone="UTC")

'2026-01-02 05:00:00'